# Semantischer Kernel

In diesem Codebeispiel verwenden Sie das [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI-Framework, um einen einfachen Agenten zu erstellen.

Das Ziel dieses Beispiels ist es, Ihnen die Schritte zu zeigen, die wir später in den weiteren Codebeispielen verwenden werden, wenn wir die verschiedenen agentischen Muster implementieren.


## Importieren Sie die benötigten Python-Pakete


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Erstellen des Clients

In diesem Beispiel verwenden wir [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) für den Zugriff auf das LLM.

Die `ai_model_id` ist als `gpt-4o-mini` definiert. Versuchen Sie, das Modell auf ein anderes Modell im GitHub Models-Marktplatz zu ändern, um die unterschiedlichen Ergebnisse zu sehen.

Um das `Azure Inference SDK` zu nutzen, das für die `base_url` von GitHub Models verwendet wird, verwenden wir den `OpenAIChatCompletion`-Connector innerhalb von Semantic Kernel. Es gibt auch andere [verfügbare Connectoren](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), um Semantic Kernel mit anderen Modellanbietern zu verwenden.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Erstellen des Agenten

Im Folgenden erstellen wir den Agenten namens `TravelAgent`.

Für dieses Beispiel verwenden wir sehr einfache Anweisungen. Sie können diese Anweisungen ändern, um zu sehen, wie der Agent unterschiedlich reagiert.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Ausführen des Agenten

Nun können wir den Agenten ausführen, indem wir einen Thread vom Typ `ChatHistoryAgentThread` definieren. Alle erforderlichen Systemnachrichten werden dem `messages`-Keyword-Argument von `invoke_stream` des Agenten übergeben.

Nachdem diese definiert sind, erstellen wir eine `user_inputs`, die das darstellt, was der Benutzer an den Agenten sendet. In diesem Fall haben wir diese Nachricht auf `Plan me a sunny vacation` gesetzt.

Fühlen Sie sich frei, diese Nachricht zu ändern, um zu sehen, wie der Agent unterschiedlich reagiert.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, weisen wir darauf hin, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
